In [1]:
hateXplain = pd.read_json("/home/mariannettrd/code/irismarechal1997/moder_ia/raw_data/230911_HateXplain.json")

hateXplain_inversed = hateXplain.transpose()

hateXplain=hateXplain_inversed


hateXplain["source"]="230911_HateXplain" # add source of the doc


hateXplain = hateXplain[["post_tokens", "annotators", "source"]].copy()

hateXplain.reset_index(inplace=True, drop=True)

hateXplain["offensive"]=0



NameError: name 'pd' is not defined

In [ ]:
hateXplain.head()

In [ ]:
# for i in range(0,len(hateXplain)): 
#     hateXplain.iloc[i,-1]= hateXplain.iloc[i,1][0]["label"]

In [ ]:
hateXplain["offensive"]=hateXplain["annotators"].apply(lambda x:x[0]["label"]).apply(lambda x:0 if x =="normal" else 1)
hateXplain["offensive"]

In [ ]:
hateXplain

## ML for multilabelling

In [2]:
import pandas as pd
import numpy as np
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import hamming_loss

In [3]:
file_path = "/home/mariannettrd/code/irismarechal1997/moder_ia/data/data_classif.csv"
df = pd.read_csv(file_path)

In [4]:
df.columns

Index(['Unnamed: 0', 'text', 'hate_speech_score', 'target_race',
       'target_religion', 'target_origin', 'target_gender_women',
       'target_gender_without_women', 'target_sexuality', 'target_age',
       'target_disability'],
      dtype='object')

In [5]:
df.drop(columns="Unnamed: 0", inplace= True)


In [6]:
df.drop(columns ="hate_speech_score", inplace = True)

In [7]:
df

,text,target_race,target_religion,target_origin,target_gender_women,target_gender_without_women,target_sexuality,target_age,target_disability
0,Yes indeed. She sort of reminds me of the elde...,True,False,False,False,False,False,False,False
1,The trans women reading this tweet right now i...,False,False,False,False,True,False,False,False
2,Question: These 4 broads who criticize America...,False,False,True,False,False,False,False,False
3,It is about time for all illegals to go back t...,False,False,True,False,False,False,False,False
4,For starters bend over the one in pink and kic...,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...
135551,عاجل سماحة #السيد_عبدالملك_بدرالدين_الحوثي نص...,False,True,False,False,False,False,False,False
135552,Millions of #Yemen-is participated in mass ral...,True,True,False,False,False,False,False,False
135553,@AbeShinzo @realDonaldTrump @shinzoabe 独裁者は行きま...,False,True,True,False,False,False,False,False
135554,Millions of #Yemen-is participated in mass ral...,False,True,False,False,False,False,False,False


In [8]:
dictrename = {'target_race': 'racism',
              'target_religion': 'religion',
              'target_origin': 'xenophobia',
              'target_gender_women':'misogyny',
              'target_gender_without_women':'transphobia',
              'target_sexuality': 'homophobia',
              'target_age': 'ageism',
              'target_disability':'validism'} 

In [9]:
df.rename(columns={'target_race': 'racism',
              'target_religion': 'religion',
              'target_origin': 'xenophobia',
              'target_gender_women':'misogyny',
              'target_gender_without_women':'transphobia',
              'target_sexuality': 'homophobia',
              'target_age': 'ageism',
              'target_disability':'validism'}, inplace = True )

In [10]:
df

,text,racism,religion,xenophobia,misogyny,transphobia,homophobia,ageism,validism
0,Yes indeed. She sort of reminds me of the elde...,True,False,False,False,False,False,False,False
1,The trans women reading this tweet right now i...,False,False,False,False,True,False,False,False
2,Question: These 4 broads who criticize America...,False,False,True,False,False,False,False,False
3,It is about time for all illegals to go back t...,False,False,True,False,False,False,False,False
4,For starters bend over the one in pink and kic...,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...
135551,عاجل سماحة #السيد_عبدالملك_بدرالدين_الحوثي نص...,False,True,False,False,False,False,False,False
135552,Millions of #Yemen-is participated in mass ral...,True,True,False,False,False,False,False,False
135553,@AbeShinzo @realDonaldTrump @shinzoabe 独裁者は行きま...,False,True,True,False,False,False,False,False
135554,Millions of #Yemen-is participated in mass ral...,False,True,False,False,False,False,False,False


In [11]:
for row in ['racism', 'religion', 'xenophobia', 'misogyny', 'transphobia', 'homophobia', 'ageism', 'validism']:
    df[row] = df[row].replace({False: 0, True: 1})

In [12]:
df

,text,racism,religion,xenophobia,misogyny,transphobia,homophobia,ageism,validism
0,Yes indeed. She sort of reminds me of the elde...,1,0,0,0,0,0,0,0
1,The trans women reading this tweet right now i...,0,0,0,0,1,0,0,0
2,Question: These 4 broads who criticize America...,0,0,1,0,0,0,0,0
3,It is about time for all illegals to go back t...,0,0,1,0,0,0,0,0
4,For starters bend over the one in pink and kic...,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
135551,عاجل سماحة #السيد_عبدالملك_بدرالدين_الحوثي نص...,0,1,0,0,0,0,0,0
135552,Millions of #Yemen-is participated in mass ral...,1,1,0,0,0,0,0,0
135553,@AbeShinzo @realDonaldTrump @shinzoabe 独裁者は行きま...,0,1,1,0,0,0,0,0
135554,Millions of #Yemen-is participated in mass ral...,0,1,0,0,0,0,0,0


In [13]:
def cleaning_table(data):
    data = data.drop_duplicates() # Remove duplicates
    data = data.dropna(subset=['offensive']) # Remove n.a. values in columns 'Label' => check column
    return data

In [14]:

def cleaning_table(data):
    data = data.drop_duplicates() # Remove duplicates
    data = data.dropna(subset=['text']) # Remove n.a. values in columns 'Label' => check column
    return data

# Note: no need to Scale the features, Encode features, Perform cyclical engineering

def cleaning_text(sentence: str) -> str:

    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers

    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation
    tokenized_sentence = word_tokenize(sentence) ## tokenize
    stop_words = set(stopwords.words('english')) ## define stopwords

    tokenized_sentence_cleaned = [ ## remove stopwords
        w for w in tokenized_sentence if not w in stop_words
    ]

    #remove words
    removed = ["user", "rt"]
    tokenized_sentence_cleaned = [ ## remove stopwords
        w for w in tokenized_sentence if not w in removed
    ]

    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "v")
        for word in tokenized_sentence_cleaned
    ]

    cleaned_sentence = ' '.join(word for word in lemmatized)

    return cleaned_sentence

In [15]:
data_processed = cleaning_table(df)

In [16]:
data_processed["text_processed"] = data_processed["text"].apply(cleaning_text)

In [17]:
data_processed

,text,racism,religion,xenophobia,misogyny,transphobia,homophobia,ageism,validism,text_processed
0,Yes indeed. She sort of reminds me of the elde...,1,0,0,0,0,0,0,0,yes indeed she sort of remind me of the elder ...
1,The trans women reading this tweet right now i...,0,0,0,0,1,0,0,0,the trans women read this tweet right now be b...
2,Question: These 4 broads who criticize America...,0,0,1,0,0,0,0,0,question these broads who criticize america wh...
3,It is about time for all illegals to go back t...,0,0,1,0,0,0,0,0,it be about time for all illegals to go back t...
4,For starters bend over the one in pink and kic...,0,0,0,1,0,0,0,0,for starters bend over the one in pink and kic...
...,...,...,...,...,...,...,...,...,...,...
135550,@AbeShinzo @realDonaldTrump @shinzoabe 独裁者は行きま...,1,1,0,0,0,0,0,0,abeshinzo realdonaldtrump shinzoabe 独裁者は行きますこれ...
135551,عاجل سماحة #السيد_عبدالملك_بدرالدين_الحوثي نص...,0,1,0,0,0,0,0,0,عاجل سماحة السيدعبدالملكبدرالدينالحوثي نصره ال...
135552,Millions of #Yemen-is participated in mass ral...,1,1,0,0,0,0,0,0,millions of yemenis participate in mass rally ...
135553,@AbeShinzo @realDonaldTrump @shinzoabe 独裁者は行きま...,0,1,1,0,0,0,0,0,abeshinzo realdonaldtrump shinzoabe 独裁者は行きますこれ...


In [18]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, hamming_loss

ModelsPerformance = {}

def metricsReport(modelName, test_labels, predictions):
    accuracy = accuracy_score(test_labels, predictions)

    macro_precision = precision_score(test_labels, predictions, average='macro')
    macro_recall = recall_score(test_labels, predictions, average='macro')
    macro_f1 = f1_score(test_labels, predictions, average='macro')

    micro_precision = precision_score(test_labels, predictions, average='micro')
    micro_recall = recall_score(test_labels, predictions, average='micro')
    micro_f1 = f1_score(test_labels, predictions, average='micro')
    hamLoss = hamming_loss(test_labels, predictions)
    print("------" + modelName + " Model Metrics-----")
    print("Accuracy: {:.4f}\nHamming Loss: {:.4f}\nPrecision:\n  - Macro: {:.4f}\n  - Micro: {:.4f}\nRecall:\n  - Macro: {:.4f}\n  - Micro: {:.4f}\nF1-measure:\n  - Macro: {:.4f}\n  - Micro: {:.4f}"\
          .format(accuracy, hamLoss, macro_precision, micro_precision, macro_recall, micro_recall, macro_f1, micro_f1))
    ModelsPerformance[modelName] = micro_f1

In [20]:
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, hamming_loss
from sklearn.pipeline import make_pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split

X = data_processed["text"]
y = data_processed.drop(labels=["text_processed", "text"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

In [21]:
# Pipeline vectorizer + LinearSVC
pipeline = make_pipeline(
    TfidfVectorizer(), 
    OneVsRestClassifier(LinearSVC(), n_jobs=-1))

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
svmPreds = pipeline.predict(X_test)
metricsReport("SVC Sq. Hinge Loss", y_test, svmPreds)


/home/mariannettrd/.pyenv/versions/3.10.6/envs/moder_ia/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/mariannettrd/.pyenv/versions/3.10.6/envs/moder_ia/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/mariannettrd/.pyenv/versions/3.10.6/envs/moder_ia/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/mariannettrd/.pyenv/versions/3.10.6/envs/moder_ia/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change f

------SVC Sq. Hinge Loss Model Metrics-----
Accuracy: 0.3629
Hamming Loss: 0.1214
Precision:
  - Macro: 0.6160
  - Micro: 0.6725
Recall:
  - Macro: 0.4714
  - Micro: 0.5517
F1-measure:
  - Macro: 0.5258
  - Micro: 0.6061


In [22]:
from sklearn.naive_bayes import MultinomialNB
pipeline = make_pipeline(
    TfidfVectorizer(), 
    OneVsRestClassifier(MultinomialNB()))

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
svmPreds = pipeline.predict(X_test)
metricsReport("Multinomial NB", y_test, svmPreds)

------Multinomial NB Model Metrics-----
Accuracy: 0.0949
Hamming Loss: 0.1563
Precision:
  - Macro: 0.5451
  - Micro: 0.7547
Recall:
  - Macro: 0.0801
  - Micro: 0.1140
F1-measure:
  - Macro: 0.1362
  - Micro: 0.1981


In [23]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
#import tensorflow as tf
#import keras
from tensorflow.keras.utils import pad_sequences

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
# import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

2023-09-18 12:19:17.569011: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-18 12:19:17.600921: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-18 12:19:17.601672: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-18 12:19:18.420610: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# Split into training and testing data
X = data_processed["text"]
y = data_processed.drop(labels=["text_processed", "text"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

### Let's tokenize the vocabulary
tk = Tokenizer()
tk.fit_on_texts(X_train)
vocab_size = len(tk.word_index)

# We apply the tokenization to the train and test set
X_train_token = tk.texts_to_sequences(X_train)
X_test_token = tk.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_token, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_token, dtype='float32', padding='post')

# Size of your embedding space = size of the vector representing each word
embedding_size = 50


# Create the model
model = Sequential()
model.add(layers.Embedding(
    input_dim=vocab_size+1, # size of the input, impacting the number of weights in the linear combinations of the neurons of the first layer
    output_dim=embedding_size, # 100
    mask_zero=True, # Built-in masking layer
))

model.add(layers.GRU(20, return_sequences=True, activation="tanh"))
model.add(layers.GRU(20, activation="tanh"))
model.add(layers.Dense(8, activation="sigmoid"))
model.summary()

model.compile(loss='binary_crossentropy',
            optimizer='rmsprop',
            metrics=['accuracy'])


es = EarlyStopping(patience=4, restore_best_weights=True)

model.fit(X_train_pad, y_train,
        epochs=5,
        batch_size=128,
        callbacks=[es]
        )

res = model.evaluate(X_test_pad, y_test)
print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          2178550   
                                                                 
 gru (GRU)                   (None, None, 20)          4320      
                                                                 
 gru_1 (GRU)                 (None, 20)                2520      
                                                                 
 dense (Dense)               (None, 8)                 168       
                                                                 
Total params: 2185558 (8.34 MB)
Trainable params: 2185558 (8.34 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
122/339 [=========>....................] - ETA: 27s - loss: 0.4720 - accuracy: 0.2288

In [ ]:
from keras.layers import Embedding

# Split into training and testing data
X = data_processed["text"]
y = data_processed.drop(labels=["text_processed", "text"], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)

### Let's tokenize the vocabulary
tk = Tokenizer()
tk.fit_on_texts(X_train)
vocab_size = len(tk.word_index)

# We apply the tokenization to the train and test set
X_train_token = tk.texts_to_sequences(X_train)
X_test_token = tk.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_token, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_token, dtype='float32', padding='post')

# Size of your embedding space = size of the vector representing each word
embedding_size = 50

# create the model
max_review_length = 600

embedding_vector_length = 32
cnn_model = Sequential()


cnn_model.add(Embedding(input_dim=vocab_size+1, output_dim=embedding_size))   
cnn_model.add(layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
cnn_model.add(layers.MaxPooling1D(pool_size=2))
cnn_model.add(layers.LSTM(100))
cnn_model.add(layers.Dense(8, activation='sigmoid'))

# Students will be ending their code here

cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(cnn_model.summary())

# Change the number of epochs and the batch size depending on the RAM Size

es = EarlyStopping(patience=4, restore_best_weights=True)

history_c =model.fit(X_train_pad, y_train,
        epochs=5,
        batch_size=128,
        callbacks=[es]
        )
res = model.evaluate(X_test_pad, y_test)
print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

In [ ]:
train_predictions = model.predict(X_train_pad)
test_predictions = model.predict(X_test_pad)

In [ ]:
train_predictions = np.argmax(train_predictions, axis=1)
test_predictions = np.argmax(test_predictions, axis=1)

In [ ]:
from sklearn.metrics import hamming_loss


treshold = 0.5

train_hamming_loss = hamming_loss(X_train_pad, (train_predictions > threshold))
test_hamming_loss = hamming_loss(X_test_pad, (test_predictions > threshold))



## One model